Use latent representation to solve auxiliary problems.

Projecting data back into the latent space.

The discriminator D discriminates not only in data space (x versus G(z)) but jointly in data and latent space ((x,E(x)) versus (G(z),z)).

$$\min_{G,E}\max_{D} V(D,E,G)$$

Code see ALI.

Change stochastic z into a deterministic one.

$$V(D,E,G)=\mathbb{E}_{x\sim p_x}[\mathbb{E}_{z\sim p_E(.|x)}\log D(x,z)]+\mathbb{E}_{z\sim p_z}\left[\mathbb{E}_{x\sim p_G(.|z)}\log(1-D(x,z))\right]$$